Possible final dataset

In [12]:
import pandas as pd
import numpy as np
import julian
import datetime as dt
import sqlite3

ModuleNotFoundError: No module named 'julian'

In [13]:
#Take a look at all the fires.  This may take some time.
con = sqlite3.connect("kaggle_dataset.sqlite")
df = pd.read_sql_query("SELECT * from Fires", con)
con.close();


NameError: name 'sqlite3' is not defined

In [6]:
size_cols = ['FIRE_SIZE_CLASS', 'DISCOVERY_DATE', 'DISCOVERY_TIME', 'CONT_DATE', 'CONT_TIME']
df = df.dropna(subset=size_cols) # remove rows where both of these are missing
df.shape

NameError: name 'df' is not defined

In [ ]:
#Build up datasets 

In [ ]:
#To make these dates and times easier to manage, let's convert them to datetime. We can add new columns DISCOVERY_DATETIME and CONTAINMENT_DATETIME.
df['DISCOVERY_DATETIME'] = df['DISCOVERY_DATE'];
df['CONT_DATETIME'] = df['CONT_DATE'];

In [ ]:
#To populate those two rows, let's convert them into datetime.
df['DISCOVERY_DATETIME'] = df['DISCOVERY_DATETIME'].apply(lambda x: julian.from_jd(x, fmt="jd"))
df['CONT_DATETIME'] = df['CONT_DATETIME'].apply(lambda x:julian.from_jd(x, fmt="jd"))


In [ ]:
#Let's also add the time 
temp_df = pd.DataFrame();
temp_df['dt'] = df['DISCOVERY_TIME'].apply(lambda x: dt.timedelta(hours=int(x[0:2]), minutes=int(x[2:5])))
df['DISCOVERY_DATETIME'] = df['DISCOVERY_DATETIME'] + temp_df['dt']
df['DISCOVERY_DATETIME'].head()

In [ ]:
#Do the same thing for CONT_DATETIME
temp_df = pd.DataFrame();
temp_df['dt'] = df['CONT_TIME'].apply(lambda x: dt.timedelta(hours=int(x[0:2]), minutes=int(x[2:5])))
df['CONT_DATETIME'] = df['CONT_DATETIME'] + temp_df['dt']
df['CONT_DATETIME'].head()

In [ ]:
df.to_csv("data_with_target.csv")
#Checkpoint to save the data with the above columns before reloading

In [3]:
df = pd.read_csv("data_with_target.csv")

FileNotFoundError: [Errno 2] File b'data_with_target.csv' does not exist: b'data_with_target.csv'

In [ ]:
dict(zip(df.STAT_CAUSE_CODE,df.STAT_CAUSE_DESCR))

In [ ]:
df['CONT_DATETIME'] = df['CONT_DATETIME'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['DISCOVERY_DATETIME'] = df['DISCOVERY_DATETIME'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['TIME_TO_CONT'] = 0;
df['TIME_TO_CONT'] = df['CONT_DATETIME'] - df['DISCOVERY_DATETIME']

In [ ]:
#Save intermediate copy
df.to_pickle("data_with_target.pkl")

In [ ]:
df.columns.values

In [ ]:
#Let's take out only the features we can use.  Last column is the labels.

In [ ]:
x = df[['DISCOVERY_DATETIME', 'STATE', 'TIME_TO_CONT', 'FIRE_YEAR', 'STAT_CAUSE_CODE', 'LATITUDE', 'LONGITUDE', 'FIRE_SIZE_CLASS']]

In [ ]:
x.head()

In [ ]:
#Convert discovery datetime to month of year
x['DISCOVERY_DATETIME'] = x['DISCOVERY_DATETIME'].apply(lambda x: x.month)


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
st =  enc.fit_transform(np.array(x['STATE']).reshape(-1,1))
x['STATE'] = pd.DataFrame(st)

In [ ]:
x.head()

In [ ]:
x['TIME_TO_CONT'] = x['TIME_TO_CONT'].apply(lambda x: x.total_seconds()/3600)


In [ ]:
x.head()

In [ ]:
di = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7}
x['FIRE_SIZE_CLASS'] = x['FIRE_SIZE_CLASS'].map(di)

In [ ]:
x.head()

Final dataset from this file:
features:
DISCOVERY_DATETIME: month of the year
STATE: ordinal-encoded state
TIME_TO_CONT: hours to containment
FIRE_YEAR: it's the year
STAT_CAUSE_CODE: fire cause
LATITUDE/LONGITUDE

label:
FIRE_SIZE_CLASS 

In [ ]:
x.to_pickle('final_dataset_1.pkl')